In [1]:
!pip install --upgrade snowflake-sqlalchemy

     |████████████████████████████████| 1.6 MB 59.0 MB/s eta 0:00:01
     |████████████████████████████████| 566 kB 90.2 MB/s eta 0:00:01


In [5]:
#!/usr/bin/env python
from sqlalchemy import create_engine
import os

os.environ["SQLALCHEMY_SILENCE_UBER_WARNING"]="1"

engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user= os.getenv('SNOWFLAKE_USERNAME'),
        password= os.getenv('SNOWFLAKE_PWD'),
        account_identifier=os.getenv('SNOWFLAKE_ACCOUNT'),
    )
)
try:
    connection = engine.connect()
    results = connection.execute('select current_version()').fetchone()
    print(results[0])
finally:
    connection.close()
    engine.dispose()

7.21.1


In [6]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

engine = create_engine(URL(
    user= os.getenv('SNOWFLAKE_USERNAME'),
    password= os.getenv('SNOWFLAKE_PWD'),
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    database = 'SNOWFLAKE_SAMPLE_DATA',
    schema = 'TPCH_SF1',
    warehouse = 'COMPUTE_WH',
    role='Public',
))

try:
    connection = engine.connect()
    results = connection.execute('SELECT C_CUSTKEY, C_NAME FROM CUSTOMER').fetchone()
    print(results[0])
finally:
    connection.close()
    engine.dispose()

30001


In [7]:
# let us create a database and table and insert some data
import pandas as pd

engine = create_engine(URL(
    user= os.getenv('SNOWFLAKE_USERNAME'),
    password= os.getenv('SNOWFLAKE_PWD'),
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    database = 'SNOWFLAKE_SAMPLE_DATA',
    schema = 'TPCH_SF1',
    warehouse = 'COMPUTE_WH',
    role='ACCOUNTADMIN',
))


In [8]:

try:
    connection = engine.connect()
    connection.execute("USE WAREHOUSE TINY_WAREHOUSE_MG")
    connection.execute("CREATE DATABASE IF NOT EXISTS testdb") # create a new database
    connection.execute("USE DATABASE testdb")
    connection.execute("CREATE SCHEMA IF NOT EXISTS testschema") # add a schema
    connection.execute("USE SCHEMA testschema")
    
    #create a table
    connection.execute(
    "CREATE OR REPLACE TABLE "
    "test_table(col1 integer, col2 string)")
    # insert some rows
    connection.execute(
    "INSERT INTO test_table(col1, col2) "
    "VALUES(123, 'test string1'),(456, 'test string2')")
    
#     #lets query this data now
#     for (col1, col2) in connection.execute("SELECT col1, col2 FROM test_table"):
#         print('{0}, {1}'.format(col1, col2))

    df = pd.read_sql_query("SELECT * FROM testdb.testschema.test_table", engine)
finally:
    connection.close()
    engine.dispose()

df.head()

,col1,col2
0,123,test string1
1,456,test string2
